In [1]:
import flask
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlalchemy

In [ ]:
# serve.py

from flask import Flask, redirect, url_for, request
from flask import render_template

engine = create_engine("sqlite:///project.db")
# creates a Flask application, named app
app = Flask(__name__)

# a route where we will display a welcome message via an HTML template
@app.route("/")
def init():  
    return render_template('table.html', x=0)


@app.route('/actors')
def asb():
    return str(engine.execute("select * from Bedding Where beds=3").fetchall());

@app.route('/xyz', methods = ['POST', 'GET'])
def xyz():
    if request.method == 'POST':
        x = request.form["x"]

        return redirect(url_for('abc', x=x,y=y,z=z))
    
@app.route('/predefined', methods = ['POST', 'GET'])
def select():
    if request.method == 'POST':
        x = request.form["query_nr"]
        return redirect(url_for('table', x=x))
    
@app.route('/abc/<x>/<y>/<z>')
def abc(x,y,z):
    return render_template('index.html', message=str(engine.execute("select * from "+x+" Where "+y+" = "+z).fetchall()))

@app.route("/table/<x>")
def table(x):
    list_of_predefined = ["",
                          "select * from Described_Description WHere square_feet=75", 
                          "select * from Reviewer WHere reviewer_name='Maria'", 
                          "select * from Room"] 
    test = engine.execute(list_of_predefined[int(x)]).fetchall();
    columns=engine.execute(list_of_predefined[int(x)]).keys();
    count=len(engine.execute(list_of_predefined[int(x)]).keys());
    return render_template('table.html', test=test, columns=columns, count= count)
# run the application
if __name__=="__main__":
     app.run(port=5000, debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Apr/2019 10:37:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2019 10:37:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2019 10:37:09] "POST /predefined HTTP/1.1" 302 -
127.0.0.1 - - [26/Apr/2019 10:37:09] "GET /table/1 HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2019 10:37:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2019 10:37:22] "GET /search HTTP/1.1" 404 -
[2019-04-26 10:37:27,218] ERROR in app: Exception on /table/0 [GET]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\engine\result.py", line 1081, in _fetchall_impl
    return self.cursor.fetchall()
AttributeError: 'NoneType' object has no attribute 'fetchall'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "C